In [1]:
import sys
!{sys.executable} -m pip install gdown h5py

In [2]:
from utils import load_it_data, visualize_img
import matplotlib.pyplot as plt
import numpy as np
import gdown
url = "https://drive.google.com/file/d/1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS/view?usp=share_link"
output = "IT_data.h5"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS
From (redirected): https://drive.google.com/uc?id=1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS&confirm=t&uuid=c76db502-df83-40e0-9fc7-b9b7ce0e5c17
To: /home/jupyter/IT_data.h5
100%|██████████| 384M/384M [00:13<00:00, 29.4MB/s] 


'IT_data.h5'

### Part 2: Predict the neural activity with the task-driven modeling approach

As you have seen in the class, the underlying hypothesis of task-driven modeling is that training the network to perform a relevant behavioral task makes the network to develop representations that resemble the ones of the biological brain. Let's test this hypothesis by loading a pre-trained ResNet50 model and use the activations of each layer to predict the neural activity. Follow these steps:

- Give as input to the network the stimuli and extract the corresponding activations of the following layers ['conv1','layer1','layer2','layer3','layer4','avgpool']
- Compute the 1000 PCs for each layer activation. (Careful that you don't want to store all activations together at the same time because it won't fit in the memory. Therefore, compute the activations and corresponding PCs for each layer and store only the computed PCs).
- Use the PCs of each layer to predict the neural activity using the linear regression models you developed before.
- Compute the goodness of fit using the correlation and explained variance metrics. Do you predict the neural activity better than before?
- Plot the distribution of explained variance with respect to the layer of the network (order them based on the depth). How does the neural activity changes across the model layers, can you make some statements about it?
- Compare the predictions that you obtained using one layer of the pretrained model and the one obtained using the same layer but from a randomly initialized model. Which network can better predict the neural activity and why?

In [3]:
from utils import load_it_data, visualize_img
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.preprocessing import StandardScaler
import torch
from torch import Tensor
from torchvision.models import ResNet, resnet50, ResNet50_Weights
import pickle
import gc
from tqdm import tqdm
import pandas as pd

In [4]:
path_to_data = '' ## Insert the folder where the data is, if you download in the same folder as this notebook then leave it blank

stimulus_train, stimulus_val, stimulus_test, objects_train, objects_val, objects_test, spikes_train, spikes_val = load_it_data(path_to_data)
layers = ["conv1", "layer1", "layer2", "layer3", "layer4", "avgpool"]

In [5]:
def apply_PCA(activations, n_components = 1000) :
    """apply PCA on the activations of a layer

    Args:
        layer_file (string): name of the layer where the activations data are extracted
        n_components (int): number of components we want to keep

    Returns:
        activations: computed PC from the activations
    """
    pca = PCA(n_components=n_components)
    pca.fit(activations)
    activations = pca.transform(activations)
    return activations

In [ ]:
def extract_activations(self, stimuli: Tensor, layer) :
    """extract the activations of the model for the given stimuli and layer

    Args:
        model (model): model we want to extract the activations from
        stimuli (ndarray): input data of the processed image's pixels

    Returns:
        dict: dictionary containing the activations for each layer of the model
    """    
    activations = []
    for x in tqdm(stimuli) : 
        x = self.conv1(x.unsqueeze(0))
        if layer == 'conv1' : 
            activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
            continue
        
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        if layer == 'layer1' : 
            activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
            continue
    
        x = self.layer2(x)
        if layer == 'layer2' : 
            activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
            continue
        
        x = self.layer3(x)
        if layer == 'layer3' : 
            activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
            continue
    
        x = self.layer4(x)
        if layer == 'layer4' :       
            activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
            continue
        
        x = self.avgpool(x)
        if layer == 'avgpool' : 
            activations.append(torch.flatten(x.squeeze(0)).detach().numpy())
            continue
    return activations

ResNet.extract_activations = extract_activations

In [15]:
# load the pre-trained ResNet50 model
stimulus_train = torch.tensor(stimulus_train)
model = resnet50(weights=ResNet50_Weights.DEFAULT) # include_top = False?
model.eval()
print()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [16]:
#Preprocess the stimuli (already done)
"""preprocess = weights.transforms()
img_transformed = preprocess(stimuli)
"""

'preprocess = weights.transforms()\nimg_transformed = preprocess(stimuli)\n'

In [17]:
# extract the activations of the layers and apply PCA on each layer to store the first 1000PCs
layers = ["conv1", "layer1", "layer2", "layer3", "layer4", "avgpool"]
for layer in layers : 
    with open(layer+'.pkl','wb') as f:
        pickle.dump(apply_PCA(model.extract_activations(stimulus_train, layer)), f)

  0%|          | 0/2592 [00:00<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'

In [ ]:
def compute_scores(model, dataset, true_prediction) : 
    """compute the r2 correlation coefficient and explained variance for each neuron individually

    Args:
        model : ML model we want to evaluate
        dataset (ndarray): input data of the processed image's pixels. Used to compute the prediction
        true_prediction (_type_): actual neural activity of the neuron

    Returns:
        tuple : r2 correlation coefficient and explained variance arrays for each neuron
    """
    predict = model.predict(dataset)
    r2 = r2_score(true_prediction, predict, multioutput = 'raw_values')
    var = explained_variance_score(true_prediction, predict, multioutput = 'raw_values')
    return r2, var


def scores_plot(r2, var) : 
    """plot the r2 correlation coefficient and explained variance for each neuron individually

    Args:
        r2 (array): r2 coefficient computed for a given model
        var (array): explained variance computed for a given model

    Returns:
        tuple ([n0, n1, ...], bins, [patches0, patches1, ...]): scatter plot of the scores
    """
    p = plt.figure(figsize=(12,7))
    plt.scatter(np.arange(len(r2)), r2, label = 'r2 score')
    plt.scatter(np.arange(len(var)), var, label = 'explained variance')
    plt.xlabel('IT neurons')
    plt.ylabel('score')
    plt.legend()
    return p

def distribution_plot(var) :
    """histogram of the distribution of the explained variance across neurons

    Args:
        var (array of double): explained variance computed for each neuron for the model

    Returns:
        tuple ([n0, n1, ...], bins, [patches0, patches1, ...]): histogram of the distribution
    """
    p = plt.figure(figsize=(12,7))
    plt.hist(var, bins=20)
    plt.xlabel('explained variance')
    plt.ylabel('count')
    return p

def evaluate_prediction_plot(model, dataset, true_prediction, model_title=None) :
    """compute and plot the r2 coefficient and explained variance for each neuron and the distribution of the explained variance

    Args:
        model : ML model we want to evaluate
        dataset (ndarray): input data of the processed image's pixels. Used to compute the prediction
        true_prediction (array): actual neural activity of the neuron 
        model_title (string, optional): description of the model used, to add at the end of the title of the graphs. Defaults to None.
    """
    
    r2, var = compute_scores(model, dataset, true_prediction)
    
    scatter_plot = scores_plot(r2, var)
    if model_title is None:
        scatter_plot.suptitle('R2 score and explained variance on the validation set for each IT neurons')
    else:
        scatter_plot.suptitle('R2 score and explained variance on the validation set for each IT neurons when fitting a {}'.format(model_title))
    
    dist_plot = distribution_plot(var)
    if model_title is None:
        dist_plot.suptitle('Histogram of the explained variance across neurons')
    else :
        dist_plot.suptitle('Histogram of the explained variance across neurons when fitting a {}'.format(model_title))

In [ ]:
# Use the PCs of each layer to predict the neural activity using linear regression models
models = {}
for layer in tqdm(layers) : 
    with open(layer+'.pkl', 'rb') as f:
        activations = pickle.load(f)  
    model = LinearRegression()
    model.fit(activations, spikes_train)
    models[layer] = model

# evaluate the models on the validation set
for layer in layers :
    evaluate_prediction_plot(models[layer], models[layer].predict(flat_val), spikes_val, model_title='linear regression on the activations of the {} layer'.format(layer))
plt.show()
    
# Compute the goodness of fit using the correlation and explained variance metrics. Do you predict the neural activity better than before?
# Plot the distribution of explained variance with respect to the layer of the network (order them based on the depth). How does the neural activity changes across the model layers, can you make some statements about it?
# Compare the predictions that you obtained using one layer of the pretrained model and the one obtained using the same layer but from a randomly initialized model. Which network can better predict the neural activity and why?